In [1]:
import textgrid
import re
from pydub import AudioSegment
from pathlib import Path
from sanitize_filename import sanitize
from tqdm.notebook import tqdm
from unicodedata import normalize
from collections import defaultdict
from typing import List

C:\Users\User\anaconda3\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
def get_tier(textgrid: textgrid.textgrid.TextGrid, name: str) -> textgrid.textgrid.IntervalTier:
    for tier in textgrid:
        if tier.name == name:
            return tier

In [3]:
def zip_(
    target_tier: textgrid.textgrid.IntervalTier,
    tiers: List[textgrid.textgrid.IntervalTier],
    skip_empty_targets=True
):
    zipped = []
    
    for interval in target_tier:
        if not interval.mark and skip_empty_targets:
            continue
        prezipped = defaultdict(list)
        for tier in tiers:
            if tier == target_tier:
                continue
            for interval_ in tier:
                if (
                    (interval.minTime >= interval_.minTime
                    and interval.maxTime <= interval_.maxTime)
                    or (interval.minTime <= interval_.minTime
                    and interval.maxTime >= interval_.maxTime)
                ):
                    prezipped[tier.name].append(interval_)
        prezipped[target_tier.name].append(interval)
        zipped.append(prezipped)
    return zipped

In [4]:
def make_prefix(value: defaultdict):
    prefix = ';'.join([f'{x} {"@".join([z.mark.strip() for z in y])}' for x, y in value.items()])
    return prefix

In [5]:
def divide(paths: List[Path], target_tier_name, folder_prefix='divided'):
    for path in tqdm(paths):
        Path(f'{folder_prefix}/' + path.name + '/sounds').mkdir(parents=True, exist_ok=True)
        files = Path(path.name).iterdir()
        wavs = [file for file in files if file.suffix == '.wav']
        tgs = [Path(wav.with_suffix('.TextGrid')) for wav in wavs]
        
        for wav, tg in zip(wavs, tgs):
            audio = AudioSegment.from_wav(wav)
            tg = textgrid.TextGrid.fromFile(tg)
            target_tier = get_tier(tg, target_tier_name)
            tiers = tg.tiers

            for tier_values in zip_(target_tier, tiers):
                start = tier_values[target_tier_name][0].minTime * 1000 - 25
                end = tier_values[target_tier_name][0].maxTime * 1000 + 25
                
                prefix = make_prefix(tier_values)
                
                prefix = sanitize(prefix.replace('̄', 'ː'))
                n = 1
                name = f'{folder_prefix}/{path.name}/sounds/{prefix};{wav.stem}_{n}.wav'.replace(':', 'ː')
                while Path(normalize('NFC', name)).exists():
                    n += 1
                    name = f'{folder_prefix}/{path.name}/sounds/{prefix};{wav.stem}_{n}.wav'.replace(':', 'ː')
                sound = audio[start:end]
                sound.export(normalize('NFC', name), format='wav')

In [11]:
divide([Path('sounds')], 'translation')
# divide([Path('ue')], 'vowel')

  0%|          | 0/1 [00:00<?, ?it/s]